# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

training_file = "traffic-signs-data/train.p"
validation_file= "traffic-signs-data/valid.p"
testing_file = "traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

import numpy as np

# Number of training examples
n_train = len(train['labels'])

# Number of testing examples.
n_valid = len(valid['labels'])

# Number of testing examples.
n_test = len(test['labels'])

# What's the shape of an traffic sign image?
shape = np.shape(train['features'])
image_shape = shape[1]

# How many unique classes/labels there are in the dataset.
n_classes = np.max(train['labels'])+1

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.ticker import NullFormatter
nullfmt = NullFormatter()

# Visualizations will be shown in the notebook.
%matplotlib inline

images = [0 for idx in range(n_classes)]
for idx in range(n_train):
    images[train['labels'][idx]] = idx

row = 5
col = 9
f, axarr = plt.subplots(row, col)
for idx in range(row*col):
    x = int(idx % row)
    y = int(idx / row)
    if idx < n_classes:
        axarr[x,y].imshow(X_train[images[idx]])
    axarr[x,y].xaxis.set_major_formatter(nullfmt)
    axarr[x,y].yaxis.set_major_formatter(nullfmt)
plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.
"""
from sklearn import preprocessing

def grayscale(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

X_train_gray = grayscale(X_train)
X_valid_gray = grayscale(X_valid)
X_test_gray = grayscale(X_test)

X_train_flat = preprocessing.scale(X_train_gray.reshape((n_train, -1)))
X_valid_flat = preprocessing.scale(X_valid_gray.reshape((n_valid, -1)))
X_test_flat = preprocessing.scale(X_test_gray.reshape((n_test, -1)))

X_train = X_train_flat.reshape((n_train, image_shape, image_shape, 1))
X_valid = X_valid_flat.reshape((n_valid, image_shape, image_shape, 1))
X_test = X_test_flat.reshape((n_test, image_shape, image_shape, 1))
"""

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

IMAGE_SIZE = 32
NUM_CHANNELS = 3
NUM_CLASSES = 43

def model(images, reuse_=False):
    with tf.variable_scope('conv1', reuse=reuse_) as scope:
        with tf.device('/cpu:0'):
            kernel = tf.get_variable('weights', [3, 3, 3, 128], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable('biases', [128], tf.float32, tf.constant_initializer(0.0))
    
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope.name)
        
    with tf.variable_scope('conv2', reuse=reuse_) as scope:
        with tf.device('/cpu:0'):
            kernel = tf.get_variable('weights', [3, 3, 128, 128], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable('biases', [128], tf.float32, tf.constant_initializer(0.0))
    
        conv = tf.nn.conv2d(conv1, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope.name)
        
    pool = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    with tf.variable_scope('conv3', reuse=reuse_) as scope:
        with tf.device('/cpu:0'):
            kernel = tf.get_variable('weights', [3, 3, 128, 256], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable('biases', [256], tf.float32, tf.constant_initializer(0.0))
    
        conv = tf.nn.conv2d(pool, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name=scope.name)

    with tf.variable_scope('conv4', reuse=reuse_) as scope:
        with tf.device('/cpu:0'):
            kernel = tf.get_variable('weights', [3, 3, 256, 256], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable('biases', [256], tf.float32, tf.constant_initializer(0.0))
    
        conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.nn.bias_add(conv, biases)
        conv4 = tf.nn.relu(bias, name=scope.name)

    pool = tf.nn.max_pool(conv4, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    with tf.variable_scope('conv5', reuse=reuse_) as scope:
        with tf.device('/cpu:0'):
            kernel = tf.get_variable('weights', [3, 3, 256, 512], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable('biases', [512], tf.float32, tf.constant_initializer(0.0))
    
        conv = tf.nn.conv2d(pool, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name=scope.name)

    with tf.variable_scope('conv6', reuse=reuse_) as scope:
        with tf.device('/cpu:0'):
            kernel = tf.get_variable('weights', [3, 3, 512, 512], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable('biases', [512], tf.float32, tf.constant_initializer(0.0))
    
        conv = tf.nn.conv2d(conv5, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.nn.bias_add(conv, biases)
        conv6 = tf.nn.relu(bias, name=scope.name)

    pool = tf.nn.max_pool(conv6, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

    with tf.variable_scope('local', reuse=reuse_) as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        pool_shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [-1, pool_shape[1] * pool_shape[2] * pool_shape[3]])
        dim = reshape.get_shape()[1].value
        with tf.device('/cpu:0'):
            weights = tf.get_variable("weights", [dim, 1024], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable("biases", [1024], tf.float32, tf.constant_initializer(0.0))

        wsum = tf.matmul(reshape, weights) + biases
        local = tf.nn.relu(wsum)

    with tf.variable_scope('softmax', reuse=reuse_) as scope:
        with tf.device('/cpu:0'):
            weights = tf.get_variable("weights", [1024, NUM_CLASSES], tf.float32, tf.truncated_normal_initializer(1e-4))
            biases = tf.get_variable("biases", [NUM_CLASSES], tf.float32, tf.constant_initializer(0.0))

        logits = tf.matmul(local, weights) + biases
    return logits, conv2

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
BATCH_SIZE = 32

images = tf.placeholder(tf.float32, shape=(None, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
labels = tf.placeholder(tf.int64, shape=(None,))
logits, tf_activation = model(images)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels))
train_op = tf.train.AdamOptimizer().minimize(loss)
predict = tf.nn.softmax(logits)
correct = tf.nn.in_top_k(tf.nn.softmax(logits), labels, 1)
top5 = tf.nn.top_k(tf.nn.softmax(logits), 5)

In [ ]:
def evaluate(X, y):
    size = X.shape[0]
    count = 0
    
    sess = tf.get_default_session()
    for offset in range(0, size, BATCH_SIZE):
        end = min(offset+BATCH_SIZE, num_examples)
        batch_X, batch_y = X[offset:end], y[offset:end]
        test_correct = sess.run([correct], feed_dict={images: batch_X, labels: batch_y})
        count += np.sum(test_correct)
    accuracy = count / size
    return accuracy

In [ ]:
def precision_recall(X, y):
    size = X.shape[0]
    class_count = [0.0 for idx in range(NUM_CLASSES)]
    predict_count = [0.0 for idx in range(NUM_CLASSES)]
    correct_count = [0.0 for idx in range(NUM_CLASSES)]
        
    sess = tf.get_default_session()
    for offset in range(0, size, BATCH_SIZE):
        end = min(offset+BATCH_SIZE, num_examples)
        batch_X, batch_y = X[offset:end], y[offset:end]
        test_softmax = sess.run(predict, feed_dict={images: batch_X, labels: batch_y})
        test_predictions = np.argmax(test_softmax, 1)

        for prediction, label in zip(test_predictions, batch_y):
            class_count[label] += 1.0
            predict_count[int(prediction)] += 1.0
            if prediction == label:
                correct_count[label] += 1.0
    precision = [correct / (total_predict+1) for correct, total_predict in zip(correct_count, predict_count)]
    recall = [correct / (class_total+1) for correct, class_total in zip(correct_count, class_count)]

    xaxis = [idx for idx in range(NUM_CLASSES)]
    f, (ax1, ax2) = plt.subplots(1, 2)
    ax1.bar(xaxis, precision)
    ax1.set_ylim((0.0, 1.0))
    ax1.set_xlabel("classes")
    ax1.set_ylabel("precision")

    ax2.bar(xaxis, recall)
    ax2.set_ylim((0.0, 1.0))
    ax2.set_xlabel("classes")
    ax2.set_ylabel("recall")
    plt.show()

In [ ]:
from sklearn.utils import shuffle

EPOCHS = 1
sess = tf.Session()

with sess.as_default():
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)

    print("Training...")

    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = min(offset+BATCH_SIZE, num_examples)
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(train_op, feed_dict={images: batch_x, labels: batch_y})

        validation_accuracy = evaluate(X_valid, y_valid)
        print("Epoch {:d}, Accuracy {:.3f}".format(i+1, validation_accuracy))
    train_accuracy = evaluate(X_train, y_train)
    print("Train Accuracy {:.3f}".format(train_accuracy))
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy {:.3f}".format(test_accuracy))
    precision_recall(X_test, y_test)

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import os
import matplotlib.image as mpimg

image_name = os.listdir('test_images/')

f, axarr = plt.subplots(1, 5)
for idx in range(len(image_name)):
    image = mpimg.imread('test_images/' + image_name[idx])
    axarr[idx].imshow(image)
    axarr[idx].xaxis.set_major_formatter(nullfmt)
    axarr[idx].yaxis.set_major_formatter(nullfmt)
plt.show()

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Predict the Sign Type for Each Image
### Analyze Performance
### Top5 Softmax Probabilities

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import os
import matplotlib.image as mpimg

image_name = os.listdir('test_images/')
image_labels = [[40], [17], [26], [14], [5]]  

count = 0.0
for idx in range(len(image_name)):
    image = mpimg.imread('test_images/' + image_name[idx])
    tf_image = np.expand_dims(image, axis=0)
    test_correct, test_top5 = sess.run([correct, top5], feed_dict={images: tf_image, labels: image_labels[idx]})
    count += np.sum(test_correct)
    print(test_top5)
print("New Image - Accuracy: {0}".format(count / len(image_name)))

---

## Step 4: Visualize the Neural Network's State with Test Images

This Section is not required, but it is an additional exercise for understanding the output of a neural network's weights. While neural networks can be a great learning device, they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network, you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the network's feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1, plt_num=1):
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={images : image_input})
    featuremaps = plt_num
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")
            
import os
import matplotlib.image as mpimg

image_name = os.listdir('test_images/')  
image = mpimg.imread('test_images/' + image_name[0])
tf_image = np.expand_dims(image, axis=0)
outputFeatureMap(tf_image, tf_activation, activation_min=-1, activation_max=-1, plt_num=10)

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 